# Database Admin 101 - Lab

## Introduction 

In this lab, you'll go through the process of designing and creating a database. From there, you'll begin to populate this table with mock data provided to you.

## Objectives

You will be able to:

* Use knowledge of the structure of databases to create a database and populate it

## The Scenario

You are looking to design a database for a school that will house various information from student grades to contact information, class roster lists and attendance. First, think of how you would design such a database. What tables would you include? What columns would each table have? What would be the primary means to join said tables?

## Creating the Database

Now that you've put a little thought into how you might design your database, it's time to go ahead and create it! Start by import the necessary packages. Then, create a database called **school.sqlite**.

In [1]:
# Import necessary packages
import pandas as pd
import sqlite3

In [2]:
# Create the database school.sqlite 
conn = sqlite3.connect('school.sqlite.db')
cur = conn.cursor()

## Create a Table for Contact Information

Create a table called contactInfo to house contact information for both students and staff. Be sure to include columns for first name, last name, role (student/staff), telephone number, street, city, state, and zipcode. Be sure to also create a primary key for the table. 

In [3]:
# Your code here
cur.execute(
""" 
CREATE TABLE contactinfo(
id INTEGER PRIMARY KEY,
firstName TEXT,
lastName TEXT,
role TEXT,
telephone  TEXT,
street TEXT,
city TEXT,
state TEXT,
zipcode TEXT
)
"""
)

## Populate the Table

Below, code is provided for you in order to load a list of dictionaries. Briefly examine the list. Each dictionary in the list will serve as an entry for your contact info table. Once you've briefly investigated the structure of this data, write a for loop to iterate through the list and create an entry in your table for each person's contact info.

In [4]:
# Load the list of dictionaries; just run this cell
import pickle

with open('contact_list.pickle', 'rb') as f:
    contacts = pickle.load(f)

In [5]:
print(contacts[0].keys())

dict_keys(['firstName', 'lastName', 'role', 'telephone ', 'street', 'city', 'state', 'zipcode '])


In [12]:
# Iterate over the contact list and populate the contactInfo table here
for contact in contacts:
    cur.execute('''
        INSERT INTO contactInfo (firstName, lastName, role, telephone, street, city, state, zipcode)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?);
    ''', (contact['firstName'], contact['lastName'], contact['role'], contact['telephone '], contact['street'], contact['city'], contact['state'], contact['zipcode ']))

**Query the Table to Ensure it is populated**

In [13]:
# Your code here 
pd.read_sql(
""" 
SELECT * 
FROM contactInfo

""", conn    
)


,id,firstName,lastName,role,telephone,street,city,state,zipcode
0,1,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,06995
1,2,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938
2,3,Linda,Jacobson,staff,4049446441,479 Musgrave Street,Atlanta,GA,30303
3,4,Andrew,Stepp,student,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012
4,5,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
5,6,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
6,7,Mary,Raines,student,9075772295,3975 Jerry Toth Drive,Ninilchik,AK,99639
7,8,Ed,Lyman,student,5179695576,3478 Be Sreet,Lansing,MI,48933
8,9,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,06995
9,10,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938


## Commit Your Changes to the Database

Persist your changes by committing them to the database.

In [14]:
# Your code here
conn.commit()

## Create a Table for Student Grades

Create a new table in the database called "grades". In the table, include the following fields: userId, courseId, grade.

** This problem is a bit more tricky and will require a dual key. (A nuance you have yet to see.)
Here's how to do that:

```SQL
CREATE TABLE table_name(
   column_1 INTEGER NOT NULL,
   column_2 INTEGER NOT NULL,
   ...
   PRIMARY KEY(column_1,column_2,...)
);
```

In [16]:
# Define a CREATE TABLE statement for 'grades'
cur.execute(
""" 
CREATE TABLE IF NOT EXISTS grades (
userId INTEGER NOT NULL,
courseId INTEGER NOT NULL,
grade TEXT,

PRIMARY KEY(userId, courseId)
)
"""
)


## Remove Duplicate Entries

An analyst just realized that there is a duplicate entry in the contactInfo table! Find and remove it.

In [29]:
# Find the duplicate entry
pd.read_sql(
""" 
WITH DuplicateRows AS (
    SELECT *,
           ROW_NUMBER() OVER(PARTITION BY firstName, lastName, role, telephone, street, city, state, zipcode 
                             ORDER BY (SELECT NULL)) AS RowNum
    FROM contactinfo
)
SELECT *
FROM DuplicateRows
WHERE RowNum > 1
ORDER BY firstName, lastName, role, telephone, street, city, state, zipcode;

""", conn
)

,id,firstName,lastName,role,telephone,street,city,state,zipcode,RowNum
0,12,Andrew,Stepp,student,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012,2
1,20,Andrew,Stepp,student,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012,3
2,9,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,06995,2
3,17,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,06995,3
4,10,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938,2
5,18,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938,3
6,16,Ed,Lyman,student,5179695576,3478 Be Sreet,Lansing,MI,48933,2
7,24,Ed,Lyman,student,5179695576,3478 Be Sreet,Lansing,MI,48933,3
8,6,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602,2
9,13,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602,3


In [24]:
# Delete the duplicate entry


In [25]:
# Check that the duplicate entry was removed


## Updating an Address

Ed Lyman just moved to `2910 Simpson Avenue York, PA 17403`. Update his address accordingly.

In [26]:
# Update Ed's address


In [27]:
# Query the database to ensure the change was made


## Commit Your Changes to the Database

Once again, persist your changes by committing them to the database.

In [28]:
# Your code here

## Summary

While there's certainly more to do with setting up and managing this database, you got a taste for creating, populating, and maintaining databases! Feel free to continue fleshing out this exercise for more practice. 